# I. Short Description

## Project presentation

The goal of this project is to practice a NoSQL database. To achieve this objective, our project aims at exploring and querying a selection of New York open data(https://nycopendata.socrata.com/data) in order to find the best place to live in this city.
To do so, we must have first have to choose a NoSQL database and load in it a set of open data which would allow us to answer to this question.


## Choice of the NoSQL database
After a quick comparison between MongoDB and Neo4j, our final choice was the document related database MongoDB. 
Indeed, several technical reasons to use MongoDB can be quoted.

1. According to a link of [Linkedin](https://www.linkedin.com/pulse/real-comparison-nosql-databases-hbase-cassandra-mongodb-sahu) some of the major advantages of MongoDB are:
* The auto sharding property which allow simplifying the scalability 
* The full index support for high performance
* The failover property which ensures its high availability.
    
2. On the same link [Linkedin](https://www.linkedin.com/pulse/real-comparison-nosql-databases-hbase-cassandra-mongodb-sahu) we can see that MongoDB is used by the company [FourSquare](https://fr.foursquare.com/).Indeed, this company is a social media allowing the users to find the nearest attractive activities using a geolocation system.
It has to be said that their problematic is similar to ours. Consequently, this observation consolidated our choice of MongoDB.

3. Otherwise,since we do not need any transactional operation in our study, hence MongoDB is sufficient for our study.

## Presentation of MongoDB

The following table was found on this link of the [tutorialspoint.com](https://www.tutorialspoint.com/mongodb/mongodb_overview.htm). It compares the terminologies of traditional relational databases (RDBMS) and those of MongoDB.

| RDBMS  |MongoDB |
|:-:|:-:|
|Database   | Database  |
|  Table |Collection   |
|  Tuple/Row |Document   |
|column|Field |
|Table Join|Embedded Documents
|Primary Key |Document|
|Primary Key |	Primary Key (Default key _id provided by mongodb itself)|


## Python and libraries
We decided to use the Python driver of MongoDB to do this project. This driver is named Pymongo and more details on it are available on this link: [api.mongodb.com](https://api.mongodb.com/python/current/).

We used the notebook Jupyter provided by the Anaconda distribution of Python are:
* [Pymongo](https://api.mongodb.com/python/current/)
* [Geopandas](http://geopandas.org/)  for geojson files
* [Geopy](https://geopy.readthedocs.io/en/1.10.0/)
* [Json](https://docs.python.org/3/library/json.html)



## Criteria for the best places to live in New York

In our view, the best place to live must have the following characteristics:
* **Located in Manhattan** which is well known to be a very attractive place for businesses. This point is an a priori information. This choice was motivated by this link: http://www.nycgo.com/boroughs-neighborhoods/manhattan.
* Close to a Hospital
* Close to a Subway station
* Has good scores in terms of energy and water consumption

To summarize the choice of our best place to live was based on "ecological aspect of the place", "its proximity to some health facilities", "its proximity to some Subway stations" and "its proximity to a large set businesses opportunities".

##  Data sets 

To search the places which have these characteristics, we downloaded the following data sets:

### Ecological aspect
Energy and water disclosure: [Energy_and_Water_Data_Disclosure_for_Local_Law__2014_.csv](https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-201/jzst-u7j8).
This data provides some information about building consumption in energy and water.

### Hospitals 
Information on the location and type of Hospitals in New York available through this data set: [Health_and_Hospitals_Corporation__HHC__Facilities.csv](https://data.cityofnewyork.us/Health/Health-and-Hospitals-Corporation-HHC-Facilities/f7b6-v6v3)

### Subway stations
Information about the location of Subway stations: [Subway Stations.geojson](https://data.cityofnewyork.us/Transportation/Subway-Stations/arq3-7z49)


## MongoDB collections and Queries

After installing, the MongoDB server, you must start it using the shell command: *service mongodb start*.
Then, we must connect to the server and create the database *NY* in which will be stored the different collections.
Globally, we created 6 collections : three of them corresponding to each of the initial data sets and three others storing the information related to the distance between the targeted addresses and Hospitals and subway stations.

### Queries workflow 

#### First analysis: on the targeted addresses

* In a first step we decided to select home located in some ecologic buildings

* And which are located in the borough **Manhattan**

* The Primary Property Type is: "Multifamily Housing" to ensure that the building is not a hotel, school etc.

* ENERGY STAR Score > 75 (This value is the threshold to get the certification according this link: www.energystar.gov/

* The total of CO² emission (Total GHG Emissions(MtCO2e)) must be low: less than 300 MtCO2e

#### Second analysis: on the hospitals
We selected the hospital located in **Manhattan** and then computed the distances between them and the targeted addresses (found in the previous section)

#### Third analysis: on the subway stations
We computed the distances between the subway stations and the targeted addresses (section a)

#### Collection containing the useful information for our objective
We then built a collection which stores the distance between the target addresses and hospitals and subway stations. Only hospital and subway located at a respective maximal distance equal to 1km and 500 meters are stored.

#### Group the result by targeted addresses

From the collection obtained in the previous section, group by targeted on the targeted homes' addresses. This allows us to find the addresses having the greatest number of choice of subway stations or hospitals.

#  II. Python Program and MongoDB queries


In [801]:
import pandas as pd
import geopandas as gp
from pymongo import MongoClient
import numpy as np
import pprint
import json
from IPython.display import display, HTML

In [802]:
import geopy

## 1) Connection to MongoDB server

This step assumes that you have already installed a MongoDB server on your computer. Once, installed, the server is launched using shell command in a terminal: *service mongodb start*. The server by default runs on the port 27017.

Then, we can connect to the server by creating a MongoDB client. Once, connected to the server, we create a databe NY where we will store each of the collections (the files downloaded for the study).

The creation of NY database is "lazy", which means that it will exist only when at least one collection containing some documents will be inserted in a its first collection.

In [803]:
client = MongoClient('localhost', 27017)
client.drop_database('NY')

In [804]:
db = client.NY
db.collection_names()

[]

## 2) Data Loadings

### a. Energy and Water database

In [840]:
EnergyWater = pd.read_csv("Energy_and_Water_Data_Disclosure_for_Local_Law__2014_.csv")
for i in EnergyWater.columns.values: print(i)

Record Number
NYC Borough, Block, and Lot (BBL)
Co-reported BBL Status
BBLs Co-reported
Reported NYC Building Identification Numbers (BINs)
Street Number
Street Name
Borough
Zip Code
BBL on the Covered Buildings List
DOF Benchmarking Submission Status
Site EUI(kBtu/ft2)
Weather Normalized Site EUI(kBtu/ft2)
Source EUI(kBtu/ft2)
Weather Normalized Source EUI(kBtu/ft2)
Municipally Supplied Potable Water - Indoor Intensity (gal/ft²)
Automatic Water Benchmarking Eligible
Reported Water Method
ENERGY STAR Score
Total GHG Emissions(MtCO2e)
Direct GHG Emissions(MtCO2e)
Indirect GHG Emissions(MtCO2e)
Reported Property Floor Area (Building(s)) (ft²)
DOF Property Floor Area (Buildngs and Parking)(ft2)
Primary Property Type - Self Selected
DOF Number of Buildings


### Hereafter, we implemented a function to clean the three columns in which we are intereste:
* ENERGY STAR Score
* Municipally Supplied Potable Water - Indoor Intensity (gal/ft²)
* Total GHG Emissions(MtCO2e)


In [842]:
# Function replacing values "Not Available", "See Primary BBL" and replace NaN by 0
# by 0 and converting strings by numerical values 

def from_string_to_num(df):
    df.fillna(0,inplace=True)
    df.replace('Not Available','0',inplace=True)
    df.replace('See Primary BBL','0',inplace=True)
    df_return = df.apply(pd.to_numeric, errors='ignore').astype(int)
    return(df_return)

In [843]:
# Converting into integer the Variable Energy star Score and Municipally Supplied Potable Water
features_to_reformat =['ENERGY STAR Score','Municipally Supplied Potable Water - Indoor Intensity (gal/ft²)',\
                       'Total GHG Emissions(MtCO2e)']
# EnergyWater[features_to_reformat]=from_string_to_num(EnergyWater[features_to_reformat])
EnergyWater[features_to_reformat] = EnergyWater[features_to_reformat] \
                                    .apply(lambda x : from_string_to_num(x))

In [854]:
# Desgin of the field "Addresses"
EnergyWater['Zip Code'].fillna(0,inplace=True)

# Converting the different elements of the address (ZipCode, StreetNumber, StreetName and Borough]
# into char in oder to concatenate them

ZipCode = EnergyWater['Zip Code'].apply(lambda x: str(np.int(x)))
StreetNumber = EnergyWater['Street Number'].fillna(0).apply(lambda x: str(x))
StreetName = EnergyWater['Street Name'].fillna(0).apply(lambda x: str(x))
Borough = EnergyWater['Borough'].fillna(0).apply(lambda x: str(x))

Adr = [(StreetNumber[i].strip() + ' ' + StreetName[i].strip()+ ' ' \
        + Borough[i].strip()+ ' ' +ZipCode[i].strip()).strip()   for i in range(len(StreetName))]

# Creating a new column "Addresses" 
EnergyWater['Addresses'] = Adr

# We are interested by 'Street Number', 'Street Name','Borough','Zip Code', 
# 'ENERGY STAR Score', and ''Municipally Supplied Potable Water - Indoor Intensity (gal/ft²)'
# We duplicate the Borough (in Addresses and Borough columns) because the column Addresses will be used
# to find the GPS coordinates and the column Borough will be used to filter the collection to select
# the borough of Manhanttan 
EW_features_of_interest =['Addresses','Total GHG Emissions(MtCO2e)', \
                          'Borough',\
                          'Primary Property Type - Self Selected','ENERGY STAR Score', \
                          'Municipally Supplied Potable Water - Indoor Intensity (gal/ft²)']

EnergyWater_useful = EnergyWater[EW_features_of_interest]

EnergyWater_useful.head(3)

,Addresses,Total GHG Emissions(MtCO2e),Borough,Primary Property Type - Self Selected,ENERGY STAR Score,Municipally Supplied Potable Water - Indoor Intensity (gal/ft²)
0,653 11 AVENUE MANHATTAN 10036,1190,MANHATTAN,Hotel,31,56
1,1050 RIVERSIDE DRIVE MANHATTAN 10032,4573,MANHATTAN,Office,20,0
2,122 WEST 27 STREET MANHATTAN 10001,698,MANHATTAN,Office,78,0


### We will store know the dataframe EnergyWater_interest as a collection into the MongoDB database NY

In [855]:
## If EnergyWaterMDB exist delete it
if db.EnergyWaterMDB.count() != 0 : 
    db.drop_collection('EnergyWaterMDB')
else:
    # Convert the dataframe of interest
    data_json = json.loads(EnergyWater_useful.to_json(orient='records'))
    db.EnergyWaterMDB.insert_many(data_json)

### First collection: EnergyWaterMDB

In [856]:
db.collection_names()

[u'HealthMDB',
 u'Subway_StationsMDB',
 u'HospitalInterestMDB',
 u'EnergyWaterMDB']

### b. Hospitals locations and type

In [813]:
Health = pd.read_csv("Health_and_Hospitals_Corporation__HHC__Facilities.csv")
for i in Health.columns.values: print(i)

Facility Type
Borough
Facility Name
Cross Streets
Phone
Location 1


We can see that the column "Location 1" is in a non convenient format. Hence, we will process it


In [816]:
Health = pd.read_csv("Health_and_Hospitals_Corporation__HHC__Facilities.csv")
Health['Location 1'].head(2)

0    79 01\nElmhurst, NY 11373\n(40.73871040256307,...
1    227 Madison Street\nNew York, NY 10002\n(40.71...
Name: Location 1, dtype: object

In [817]:
Hospt_address = []   # address
Hospt_gps_coordinate = []
temp_var = Health['Location 1'].apply(lambda x : x.split('\n'))
Hospt_address=[i[0] +' '+ i[1] for i in temp_var]

Hospt_gps_coordinate =[i[2] for i in tt]


Health['Addresses']=pd.DataFrame(Hospt_adress)
Health['GPScoord']=pd.DataFrame(Hospt_gps)
Health_Features_to_drop =['Location 1','Cross Streets','Phone']
Health_useful = Health.drop(Health_Features_to_drop, axis=1)
Health_useful.head(3)

,Facility Type,Borough,Facility Name,Addresses,GPScoord
0,Child Health Center,Queens,Elmhurst Hospital Center,"79 01 Elmhurst, NY 11373","(40.73871040256307, -73.87835115518163)"
1,Nursing Home,Manhattan,Gouverneur Healthcare Services,"227 Madison Street New York, NY 10002","(40.712841397447676, -73.98759853472393)"
2,Nursing Home,Manhattan,Coler-Goldwater Specialty Hospital and Nursing...,"1 Main St Roosevelt Island New York, NY 10044","(40.76197648622593, -73.94999967494499)"


In [818]:
## If EnergyWaterMDB exist delete it
if db.HealthMDB.count() != 0 : 
    db.drop_collection('HealthMDB')
else:
    # Convert the dataframe of interest
    data_json = json.loads(Health_useful.to_json(orient='records'))
    db.HealthMDB.insert_many(data_json)

### Second collection: HealthMDB


In [820]:
db.collection_names()

[u'HealthMDB', u'EnergyWaterMDB']

### d. Subway stations
This is a geojson file which requires geopandas to read it in pyhton

In [821]:
Subway_Stations = gp.read_file("Subway Stations.geojson")

### We want to extract GPS coordinates from the Geometry column. To do so, we implemented a function getGPS

In [822]:
def getGPS(p):
    return (p.x, p.y)

In [823]:
cs = Subway_Stations['geometry'].centroid
Subway_Stations['gpsSub'] = map(getGPS, cs)
Subway_Stations_df = pd.DataFrame(Subway_Stations)
Subway_Stations_useful = Subway_Stations_df.drop(['geometry','url'],axis=1)

In [760]:
Subway_Stations_useful.head(3)

,line,name,gpsSub
0,B-D-F-M,W 4th St - Washington Sq (Lower),"(-74.0003081471, 40.7322548265)"
1,6-6 Express,Buhre Ave,"(-73.8325689992, 40.8468103326)"
2,4-6-6 Express,51st St,"(-73.9719200001, 40.7571073331)"


In [825]:
if db.Subway_StationsMDB.count() != 0 : 
    db.drop_collection('Subway_StationsMDB')
else:
    data_json = json.loads(Subway_Stations_useful.to_json(orient='records'))
    db.Subway_StationsMDB.insert_many(data_json)

### Third collection: Subway_StationsMDB

In [826]:
db.collection_names()

[u'HealthMDB', u'Subway_StationsMDB', u'EnergyWaterMDB']

## Queries on EnergyWaterMDB to find a first set of interesting addresses

These queries are driven by three mains objectives:
* Located in the borough Manhattan 
* The Primary Property Type is: "Multifamily Housing"
* ENERGY STAR Score > 75 (This value is the threshold to get the certification according this link: [www.energystar.gov/](https://www.energystar.gov/buildings/facility-owners-and-managers/existing-buildings/use-portfolio-manager/understand-metrics/how-1-100)
* The total of CO² emission (Total GHG Emissions(MtCO2e)) is low: less than 300 MtCO2e




In [857]:
query_Home_Energy_target = {'Borough':'MANHATTAN',\
                            'ENERGY STAR Score':{'$gt' : 75},\
                            'Primary Property Type - Self Selected':'Multifamily Housing',\
                            'Total GHG Emissions(MtCO2e)':{'$lt' : 300}}


### A first set of addresses of interest

In [858]:
db.EnergyWaterMDB.find(query_Home_Energy_target).count()

171

### Let us compute the GPS coordinate of targeted houses (according to the criteria explicited above).  <span style="color: blue">This can take a wile 

In [863]:
query_Home_Energy_target = {'Borough':'MANHATTAN',\
                            'ENERGY STAR Score':{'$gt' : 75},\
                            'Primary Property Type - Self Selected':'Multifamily Housing',\
                            'Total GHG Emissions(MtCO2e)':{'$lt' : 300}}

# Adding the field "gps" to the EnergyWaterMDB
# Some of the addresses are not resolved by the api geopy so we arbitrary 
# gave a null pair (0,0) of coordinates

from geopy.geocoders import Nominatim
geolocator = Nominatim()
gps_list = list()
for i in db.EnergyWaterMDB.find(query_Home_Energy_target):
    location = geolocator.geocode(i['Addresses'])

    if location is None:
        gps_i = (0,0)
    else:
        gps_i = (location.longitude,location.latitude)
    #Updating the EnergyWaterMDB collection
    gps_list.append(gps_i)
    db.EnergyWaterMDB.update_one( {"_id":i["_id"]}, {'$set': {'GPScoord': gps_i}})

### Remove all the documents that are not of interest

In [861]:
for c in db.EnergyWaterMDB.find({'GPScoord' : { '$exists': False}}):
    db.EnergyWaterMDB.delete_one(c)

In [862]:
db.EnergyWaterMDB.find().count()

171

## Queries about the types and locations of targeted hospitals
* The **type** of hospital that interest us are: **"Acute Care Hospital" and "Diagnostic & Treatment Center"**
* Moreover they must be **located in Manhattan**.

These queries lead to a set of 5 Hospitals,which are then stored them in a collection HospitalInterestMDB

In [864]:
hospital_query_type = {"$or":[{'Facility Type':'Acute Care Hospital'},\
                          {'Facility Type':'Diagnostic & Treatment Center'}]}
hospital_query_location = {'Borough':'Manhattan'}

hospital_query = {'$and':[hospital_query_type,hospital_query_location]}

cursor_Hospital = db.HealthMDB.find(hospital_query)

hospitals_of_interest_name = []
hospitals_of_interest_coord = []
hospitals_of_interest_Ad = []

for h in cursor_Hospital:
    hospitals_of_interest_name.append(h['Facility Name'])
    hospitals_of_interest_coord.append(h['GPScoord'])
    hospitals_of_interest_Ad.append(h['Addresses'])
hospital_of_interest=pd.DataFrame({'HospitalName':hospitals_of_interest_name,\
                                   'Adresses':hospitals_of_interest_Ad,\
                                   'GPS':hospitals_of_interest_coord})
hospital_of_interest


,Adresses,GPS,HospitalName
0,"227 Madison Street New York, NY 10002","(40.712841397447676, -73.98759853472393)",Gouverneur Healthcare Services
1,"462 First Avenue New York, NY 10016","(40.73962320748018, -73.97657284664467)",Bellevue Hospital Center
2,"506 Lenox Avenue New York, NY 10037","(40.814275217501645, -73.9406028312914)",Harlem Hospital Center
3,"215 125 Street New York, NY 10027","(40.809338651191524, -73.94919748407278)",Renaissance Health Care Network Diagnostic & T...
4,"1901 First Avenue New York, NY 10029","(40.784557024104835, -73.9437956536432)",Metropolitan Hospital Center


In [865]:
## Storing the targeted Hospital
hospital_of_interest=pd.DataFrame({'HospitalName':hospitals_of_interest_name,\
                                   'Addresses':hospitals_of_interest_Ad,\
                                   'GPScoord':hospitals_of_interest_coord})
hospital_of_interest
db.drop_collection('HospitalInterestMDB')
data_json = json.loads(hospital_of_interest.to_json(orient='records'))
db.HospitalInterestMDB.insert_many(data_json)

### Fourth collection: HospitalInterestMDB

In [866]:
db.collection_names()

[u'HealthMDB',
 u'Subway_StationsMDB',
 u'HospitalInterestMDB',
 u'EnergyWaterMDB']

## Calculating the Distances between the targeted homes and the Hospitals & Subway Stations

Geopy can compute geodesic distance between two points using the Vincenty distance or great-circle distance formulas. However, to reduce the runtime we decided to calculate this distance through a function that we implemented.

In [867]:
# Function which compute the distance between two locations given their longitude and latitude
from math import sin, cos, sqrt, atan2, radians
# approximate radius of earth in km
R = 6373.0
def distance_two_locations(l1,l2):
    
    lat1 = radians(l1[1])
    lon1 = radians(l1[0])
    lat2 = radians(l2[1])
    lon2 = radians(l2[0])
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return round(distance,2)


### The goal of the next step is to compute the distance between the targeted Homes and  Hospitals. Then, we will store the result in a collection.

In [869]:
Adr_home = []
Adr_hosp = []
Dist_to_Home = []
gps_home_list = []
gps_hosp_list = []
for e_doc in db.EnergyWaterMDB.find():
    for h_doc in db.HospitalInterestMDB.find():
        gps_home = e_doc['GPScoord']
        gps_hospital = h_doc['GPScoord']
        # Reformatting gps coordinate to have the following format : tuple (longitude,latitude)
        gps_hospital = (float(gps_hospital.split(',')[1][:-1]),float(gps_hospital.split(',')[0][1:]))
        # Since the geopy library failed on the resolution of some postal addresses:
        if (gps_home != [0,0]) & (gps_hospital!= [0,0]):
            gps_home_list.append(gps_home)
            gps_hosp_list.append(gps_hospital)
            d = distance_two_locations(gps_home,gps_hospital)
            Adr_home.append(e_doc['Addresses'])
            Adr_hosp.append(h_doc['Addresses'])
            Dist_to_Home.append(d)

Home_Hospital = pd.DataFrame({'gpsHome':gps_home_list,'gpsHospital':gps_hosp_list,\
                              'HospitalName':h_doc['HospitalName'],'AddressHome':Adr_home,\
                              'AdressHospital':Adr_hosp,'DistHomeHosp':Dist_to_Home})

# Selecting the interesting columns
Home_Hospital = Home_Hospital[['AddressHome','gpsHome','gpsHospital','AdressHospital',\
                               'HospitalName','DistHomeHosp']]

Home_Hospital = Home_Hospital.sort_values('DistHomeHosp')
Home_Hospital.reset_index(drop=True, inplace=True)
# if EnergyWaterMDB exist delete it
db.drop_collection('HomeHospitalDistMDB')
data_json = json.loads(Home_Hospital.to_json(orient='records'))
db.Home_Hospital_Distance_MDB.insert_many(data_json)

### Fifth collection: HospitalInterestMDB

In [871]:
db.collection_names()

[u'HealthMDB',
 u'Subway_StationsMDB',
 u'Home_Hospital_Distance_MDB',
 u'HospitalInterestMDB',
 u'EnergyWaterMDB']

### The goal of the next step is to compute the distance between the targeted Homes and the different subway stations. 

the maximal** distances between the homes of interest** and a **hospital and a subway station**  are respectively is **1 km and 500 meters**

In [872]:
Adr_home = []
Dist_to_Home = []
gps_home_list = []
gps_sub_list = []
Dist_to_Sub =[]
Dist_to_Hosp =[]

Home_Hospital = Home_Hospital[['AddressHome','gpsHome','gpsHospital','AdressHospital',\
                               'HospitalName','DistHomeHosp']]

# Home close to a hospital with a distance less than 1km
d_hospital_max = 1
query_home = {'DistHomeHosp':{'$lt':d_hospital_max}}

# Home close to a subway station with a distance less than 0.5km
d_subway_max=0.5

for e_doc in db.Home_Hospital_Distance_MDB.find(query_home):
    for ss_doc in db.Subway_StationsMDB.find():
        loc_home = e_doc['gpsHome']
        loc_subw = ss_doc['gpsSub']
        d = distance_two_locations(loc_home,loc_subw)
        if d <= d_subway_max:
            Adr_home.append(e_doc['AddressHome'])
            Dist_to_Hosp.append(e_doc['DistHomeHosp'])

            gps_home_list.append(tuple(loc_home))
            gps_sub_list.append(tuple(loc_subw))
            Dist_to_Sub.append(d)

Home_Dist_Subway_Hospital= pd.DataFrame({'DistHosp':Dist_to_Hosp,'AdressHome':Adr_home,\
                                         'gpsHome':gps_home_list,'gpsSubway':gps_sub_list,\
                                         'DistSubway':Dist_to_Sub})

Home_Dist_Subway_Hospital = Home_Dist_Subway_Hospital[['AdressHome','gpsHome','gpsSubway',\
                                                       'DistHosp','DistSubway']]

Home_Dist_Subway_Hospital=Home_Dist_Subway_Hospital.sort_values(['DistHosp','DistSubway'])

Home_Dist_Subway_Hospital.reset_index(drop=True, inplace=True)

data_json = json.loads(Home_Dist_Subway_Hospital.to_json(orient='records'))
db.Home_Dist_Subway_Hospital_MDB.insert_many(data_json)

### Fifth collection: Home_Dist_Subway_Hospital_MDB

In [881]:
db.collection_names()

[u'HealthMDB',
 u'Subway_StationsMDB',
 u'Home_Hospital_Distance_MDB',
 u'Home_Dist_Subway_Hospital_MDB',
 u'HospitalInterestMDB',
 u'EnergyWaterMDB']

### The list of the best place to leave according to our criteria. The list is sorted by ascending order of the distance to a hospital (first) and to a subway station (second). Hence the first address "272 CHERRY STREET MANHATTAN 10002" is the one which is the closest to a hospital.

In [882]:
BestPlaceToLive = Home_Dist_Subway_Hospital.AdressHome.drop_duplicates()
BestPlaceToLive

0         272 CHERRY STREET MANHATTAN 10002
1    320 ST NICHOLAS AVENUE MANHATTAN 10027
2       95 LEXINGTON AVENUE MANHATTAN 10016
5     110 MORNINGSIDE DRIVE MANHATTAN 10027
6      90 MORNINGSIDE DRIVE MANHATTAN 10027
7          1128 PARK AVENUE MANHATTAN 10128
Name: AdressHome, dtype: object

### Let us summarize the result using a "group by" AdressHome

In [883]:
# The reducer function
from bson.code import Code
reducer = Code("""
                function(obj, prev){
                  prev.count++;
                }
                """)

results = db.Home_Dist_Subway_Hospital_MDB.group(key={"AdressHome":1},\
                                                     condition={}, initial={"count": 0}, reduce=reducer)
results

[{u'AdressHome': u'272 CHERRY STREET MANHATTAN 10002', u'count': 1.0},
 {u'AdressHome': u'320 ST NICHOLAS AVENUE MANHATTAN 10027', u'count': 1.0},
 {u'AdressHome': u'95 LEXINGTON AVENUE MANHATTAN 10016', u'count': 3.0},
 {u'AdressHome': u'110 MORNINGSIDE DRIVE MANHATTAN 10027', u'count': 1.0},
 {u'AdressHome': u'90 MORNINGSIDE DRIVE MANHATTAN 10027', u'count': 1.0},
 {u'AdressHome': u'1128 PARK AVENUE MANHATTAN 10128', u'count': 2.0}]

In [884]:
selected_streets = db.Home_Dist_Subway_Hospital_MDB.find({}).distinct('AdressHome')
selected_streets

[u'272 CHERRY STREET MANHATTAN 10002',
 u'320 ST NICHOLAS AVENUE MANHATTAN 10027',
 u'95 LEXINGTON AVENUE MANHATTAN 10016',
 u'110 MORNINGSIDE DRIVE MANHATTAN 10027',
 u'90 MORNINGSIDE DRIVE MANHATTAN 10027',
 u'1128 PARK AVENUE MANHATTAN 10128']

### Let us summarize the result using a "group by" AdressHome and sorting according to the number of occurences. Indeed, the home with the greatest number of occurences provide more choice in terms of subway stations and or hospital.

In [885]:
from bson.son import SON
pipeline = [
    {"$group": {"_id": "$AdressHome", "count": {"$sum": 1}}},
    {"$sort": SON([("count", -1), ("_id", -1)])}
 ]
pprint.pprint(list(db.Home_Dist_Subway_Hospital_MDB.aggregate(pipeline)))

[{u'_id': u'95 LEXINGTON AVENUE MANHATTAN 10016', u'count': 3},
 {u'_id': u'1128 PARK AVENUE MANHATTAN 10128', u'count': 2},
 {u'_id': u'90 MORNINGSIDE DRIVE MANHATTAN 10027', u'count': 1},
 {u'_id': u'320 ST NICHOLAS AVENUE MANHATTAN 10027', u'count': 1},
 {u'_id': u'272 CHERRY STREET MANHATTAN 10002', u'count': 1},
 {u'_id': u'110 MORNINGSIDE DRIVE MANHATTAN 10027', u'count': 1}]
